# Script for a period

In [92]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [93]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [94]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [95]:
data_giper

{'adverts': [{'type': 8,
   'status': 9,
   'count': 14,
   'advert_list': [{'advertId': 22844037,
     'changeTime': '2025-04-30T16:06:17.515678+03:00'},
    {'advertId': 22844090, 'changeTime': '2025-02-19T15:07:09.546816+03:00'},
    {'advertId': 22844946, 'changeTime': '2025-04-30T16:05:07.300432+03:00'},
    {'advertId': 22844983, 'changeTime': '2025-03-03T13:35:24.529729+03:00'},
    {'advertId': 22845720, 'changeTime': '2025-01-27T17:10:50.894875+03:00'},
    {'advertId': 22845877, 'changeTime': '2025-03-03T16:10:31.479042+03:00'},
    {'advertId': 22845971, 'changeTime': '2025-03-03T11:55:07.014918+03:00'},
    {'advertId': 22902062, 'changeTime': '2025-02-12T11:05:47.365327+03:00'},
    {'advertId': 22902096, 'changeTime': '2025-02-12T11:05:22.842103+03:00'},
    {'advertId': 22931827, 'changeTime': '2025-02-12T11:10:53.77795+03:00'},
    {'advertId': 22950186, 'changeTime': '2025-02-26T17:11:30.515678+03:00'},
    {'advertId': 22954264, 'changeTime': '2025-02-27T14:07:05.6654

In [96]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [97]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [98]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[18870701, 20397161, 20954456, 20955117, 21577733, 21577749, 21578389, 21624850, 21698516, 23231292, 24038799, 24780937, 462902, 462934, 462959, 462991, 463002, 463125, 463138, 463196, 463211, 463364, 480144, 480165, 480178, 480275, 480291, 480347, 480356, 480455, 480513, 480527, 480531, 480543, 480548, 610764, 610778, 610787, 610835, 610857, 610869, 622222, 622292, 622297, 622326, 622347, 622351, 622366, 622371, 622376], [622381, 622386, 622391, 622397, 622402, 622406, 622420, 622432, 622441, 622448, 622454, 788494, 788498, 788605, 788619, 788659, 788667, 789205, 789210, 953446, 953780, 953785, 958839, 964993, 964999, 965005, 965011, 965014, 965020, 1005723, 1320700, 1320709, 1320718, 1801052, 1801069, 1801426, 1801455, 1801541, 1801574, 1801644, 1801713, 1802589, 1802611, 2594357, 2594580, 2594976, 2595030, 2612852, 2614375, 2852177], [3467141, 3474261, 3474599, 3476612, 3476872, 4558112, 4981827, 5307465, 6626496, 6844544, 6844902, 6923850, 7202673

In [99]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-04-30,2025-04-30,Braun_Фотоэпиляторы_2 SKU_30.04.2025,25217413,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-04-30,2025-04-30,Caso_Аэрофритюрницы_30.04.2025,25214927,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-04-29,2025-04-29,Oclean_Бюджетная щетка_29.04.2025,25188804,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-04-29,2025-04-29,Elikor_Cube_3 SKU_РКП_29.04.2025,25173639,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-04-29,2025-04-29,Elikor_Cube_3 SKU_29.04.2025,25171712,Идут показы,Аукцион,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
486,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
487,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
488,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
489,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [ ]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-05-05",  # Replace with your start date
    "end": "2025-05-05"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [101]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            error_msg = f"Error for chunk {idx + 1}: {response.status_code}, {response.text}"
            print(error_msg)
            
            # Check for specific error condition
            if response.status_code == 400 and "no companies with correct intervals" in response.text:
                print("Stopping execution due to invalid interval error")
                break  # Exit the loop immediately
                
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)

# Only proceed if data was retrieved
if all_campaign_data_guten:
    # Combine all campaign data into a DataFrame
    campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

    # Save the combined data to a text file
    with open("response_data_guten.txt", "w", encoding="utf-8") as file:
        json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

    # Display the combined DataFrame
    print("Combined Campaign Data")
else:
    print("No data retrieved - check your parameters or date interval")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Stopping execution due to invalid interval error
Combined Campaign Data


### Giper

In [102]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [103]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
        response = requests.post(url, headers=headers, json=payload)
        
        time.sleep(65)  # Delay to avoid rate limits

        if response.status_code == 200:
            try:
                data = response.json()
                if data:  # Ensure data is not None or empty
                    all_data.extend(data)
                    print(f"Data retrieved successfully for chunk {idx + 1}")
                else:
                    print(f"Empty response for chunk {idx + 1}")
            except ValueError:  # Catch JSON decode errors
                print(f"Invalid JSON response for chunk {idx + 1}: {response.text}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Empty response for chunk 1


### Smart

In [104]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [109]:
all_campaign_data_smart

[{'interval': {'begin': '2025-05-04', 'end': '2025-05-04'},
  'views': 12551,
  'clicks': 710,
  'ctr': 5.66,
  'cpc': 2.36,
  'sum': 1678.81,
  'atbs': 158,
  'orders': 27,
  'cr': 3.8,
  'shks': 27,
  'sum_price': 137047,
  'days': [{'date': '2025-05-04T03:00:00+03:00',
    'views': 12551,
    'clicks': 710,
    'ctr': 0.01,
    'cpc': 2.36,
    'sum': 1678.81,
    'atbs': 158,
    'orders': 27,
    'cr': 3.8,
    'shks': 27,
    'sum_price': 137047,
    'apps': [{'views': 1057,
      'clicks': 37,
      'ctr': 3.5,
      'cpc': 3.79,
      'sum': 140.12,
      'atbs': 15,
      'orders': 3,
      'cr': 8.11,
      'shks': 3,
      'sum_price': 16266,
      'nm': [{'views': 62,
        'clicks': 3,
        'ctr': 4.84,
        'cpc': 2.99,
        'sum': 8.98,
        'atbs': 1,
        'orders': 0,
        'cr': 0,
        'shks': 0,
        'sum_price': 0,
        'name': 'Эпилятор Silk-epil 3 3-176 для сухой эпиляции с подсветкой',
        'nmId': 205531989},
       {'views': 125,

In [114]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

52


In [116]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create DataFrames
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)  # Could be empty
df_smart = pd.DataFrame(flattened_data_smart)

# Function to safely process date columns
def process_date_column(df):
    if not df.empty and 'date' in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        df["date"] = df["date"].dt.tz_localize(None)
    return df

# Apply processing ONCE (no need for the second block)
df_guten = process_date_column(df_guten)
df_giper = process_date_column(df_giper)
df_kitchen = process_date_column(df_kitchen)  # Safe even if empty
df_smart = process_date_column(df_smart)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-05-04 03:00:00,137609384,Бритва станок Fusion5 ProGlide Power Flexball ...,257,9,3.50,4.02,36.19,0,0,0.00,0,0,21497844
1,2025-05-04 03:00:00,137609312,Бритва станок Mach3 с 1 сменной кассетой,2,0,0.00,0.00,0.30,0,0,0.00,0,0,21497844
2,2025-05-04 03:00:00,137609308,Бритва станок Fusion 5 ProGlide с 1 кассетой,280,7,2.50,5.56,38.92,0,0,0.00,0,0,21497844
3,2025-05-04 03:00:00,137609340,Бритва станок Fusion 5 с 4 сменными кассетами,3,0,0.00,0.00,0.45,0,0,0.00,0,0,21497844
4,2025-05-04 03:00:00,137609389,Бритва станок Fusion 5 ProGlide с 3 кассетами,245,7,2.86,4.86,33.99,0,0,0.00,0,0,21497844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,2025-05-04 03:00:00,252275490,Вытяжка кухонная 60 см встраиваемая телескопич...,406,18,4.43,4.29,77.14,3,1,5.56,1,6331,19276556
653,2025-05-04 03:00:00,252275489,Вытяжка кухонная 60 см встраиваемая телескопич...,227,20,8.81,2.16,43.13,2,1,5.00,1,5980,19276556
654,2025-05-04 03:00:00,252275488,Вытяжка кухонная 60 см встраиваемая телескопич...,490,22,4.49,13.03,286.65,0,0,0.00,0,0,19276556
655,2025-05-04 03:00:00,252275487,Вытяжка кухонная 60 см встраиваемая телескопич...,186,6,3.23,5.89,35.34,1,0,0.00,0,0,19276556


In [118]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",
        "name": "first",
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",
        "cpc": "mean",
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"
    })
) if not df_kitchen.empty else pd.DataFrame()  # Return empty DataFrame if df_kitchen is empty

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25088481.0,2025-05-04 03:00:00,Эпилятор женский электрический Silk-epil 9-880,424.0,27.0,5.066667,5.400000,94.06,0.0,0.0,0.000000,0.0,0.0,21204285.0,WB-GutenTech,Wildberries
1,25623718.0,2025-05-04 03:00:00,Бритва мужская электрическая S3 300BT с триммером,212.0,9.0,3.350000,1.330000,22.07,1.0,1.0,5.556667,1.0,7858.0,21202453.0,WB-GutenTech,Wildberries
2,25623718.0,2025-05-04 03:00:00,Бритва мужская электрическая S3 300BT с триммером,350.0,17.0,4.900000,4.183333,71.39,2.0,0.0,0.000000,0.0,0.0,21202745.0,WB-GutenTech,Wildberries
3,25624340.0,2025-05-04 03:00:00,Электробритва Series 3 300s,446.0,13.0,2.416667,1.963333,43.68,0.0,0.0,0.000000,0.0,0.0,21202453.0,WB-GutenTech,Wildberries
4,25624340.0,2025-05-04 03:00:00,Электробритва Series 3 300s,393.0,24.0,5.460000,4.510000,80.08,5.0,2.0,6.986667,2.0,13090.0,21202745.0,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235597032.0,2025-05-04 03:00:00,Эпилятор женский электрический Silk-epil 7-030,3192.0,131.0,3.843333,3.766667,461.82,24.0,3.0,4.603333,3.0,31914.0,20822563.0,WB-Smart-Market,Wildberries
450,235599972.0,2025-05-04 03:00:00,Электробритва Series 9 Pro+ 9510s,563.0,16.0,2.960000,3.863333,63.52,3.0,2.0,10.320000,2.0,59954.0,15562363.0,WB-Smart-Market,Wildberries
451,235604574.0,2025-05-04 03:00:00,Триммер для бороды и усов BT5421,268.0,6.0,1.576667,4.930000,31.91,1.0,0.0,0.000000,0.0,0.0,19597465.0,WB-Smart-Market,Wildberries
452,235638522.0,2025-05-04 03:00:00,Эпилятор Silk-epil 5 5-000,2691.0,101.0,3.316667,4.240000,353.95,16.0,0.0,0.000000,0.0,0.0,20822563.0,WB-Smart-Market,Wildberries


In [119]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final
df_final_copy = df_final.copy()


In [120]:
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481.0,2025-05-04 03:00:00,Эпилятор женский электрический Silk-epil 9-880,424.0,27.0,94.06,0.0,0.0,0.0,0.0,21204285.0,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718.0,2025-05-04 03:00:00,Бритва мужская электрическая S3 300BT с триммером,212.0,9.0,22.07,1.0,1.0,1.0,7858.0,21202453.0,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718.0,2025-05-04 03:00:00,Бритва мужская электрическая S3 300BT с триммером,350.0,17.0,71.39,2.0,0.0,0.0,0.0,21202745.0,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340.0,2025-05-04 03:00:00,Электробритва Series 3 300s,446.0,13.0,43.68,0.0,0.0,0.0,0.0,21202453.0,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340.0,2025-05-04 03:00:00,Электробритва Series 3 300s,393.0,24.0,80.08,5.0,2.0,2.0,13090.0,21202745.0,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235597032.0,2025-05-04 03:00:00,Эпилятор женский электрический Silk-epil 7-030,3192.0,131.0,461.82,24.0,3.0,3.0,31914.0,20822563.0,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
450,235599972.0,2025-05-04 03:00:00,Электробритва Series 9 Pro+ 9510s,563.0,16.0,63.52,3.0,2.0,2.0,59954.0,15562363.0,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
451,235604574.0,2025-05-04 03:00:00,Триммер для бороды и усов BT5421,268.0,6.0,31.91,1.0,0.0,0.0,0.0,19597465.0,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
452,235638522.0,2025-05-04 03:00:00,Эпилятор Silk-epil 5 5-000,2691.0,101.0,353.95,16.0,0.0,0.0,0.0,20822563.0,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания


In [ ]:
# Define headers for each project
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set

}

# Define the API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"

# Set fixed date range for testing
yesterday_start = '2025-05-05 00:00:00'
yesterday_end = '2025-05-05 23:59:59'

def get_report(url, headers, begin, end, page, project_name):
    all_data = []
    max_retries = 3
    retry_delay = 30  # seconds
    request_timeout = 60  # seconds
    
    while True:
        for attempt in range(max_retries):
            try:
                # Define the request body with all required parameters
                request_body = {
                    "period": {
                        "begin": begin,
                        "end": end
                    },
                    "orderBy": {
                        "field": "ordersSumRub",
                        "mode": "desc"
                    },
                    "page": page,
                    "timezone": "Europe/Moscow",  # Added as recommended in API docs
                    "brandNames": [],  # Empty array to get all brands
                    "objectIDs": [],   # Empty array to get all objects
                    "nmIDs": []        # Empty array to get all items
                }

                json_data = json.dumps(request_body, ensure_ascii=False)
                
                # Send the POST request with timeout
                response = requests.post(
                    url, 
                    headers=headers, 
                    data=json_data, 
                    timeout=request_timeout
                )
                
                # Handle rate limiting (429 status code)
                if response.status_code == 429:
                    wait_time = int(response.headers.get('Retry-After', retry_delay))
                    print(f"Rate limit exceeded for {project_name}. Waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                    continue
                
                # Check for successful response
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code} for {project_name}, page {page}")
                    print("Response text:", response.text)
                    if attempt == max_retries - 1:
                        return all_data
                    time.sleep(retry_delay)
                    continue
                
                data = response.json()
                
                # Validate response structure
                if not data.get('data') or not isinstance(data['data'].get('cards'), list):
                    print(f"Invalid data structure received for {project_name}, page {page}")
                    return all_data
                
                cards = data['data']['cards']
                
                # Check if we have any data
                if not cards:
                    print(f"No more data available for {project_name}")
                    return all_data
                
                all_data.extend(cards)
                print(f"Page {page} retrieved successfully for {project_name} (got {len(cards)} items)")
                
                # Check if there are more pages (using isNextPage flag)
                if not data['data'].get('isNextPage', False):
                    print(f"Reached last page for {project_name}")
                    return all_data
                
                # Prepare for next page
                page += 1
                time.sleep(5)  # Reduced delay between pages to 5 seconds
                break  # Success, break out of retry loop
                
            except requests.exceptions.RequestException as e:
                print(f"Request failed for {project_name}, page {page}: {str(e)}")
                if attempt == max_retries - 1:
                    return all_data
                time.sleep(retry_delay)
                continue
            except Exception as e:
                print(f"Unexpected error for {project_name}, page {page}: {str(e)}")
                return all_data

# Get data for each project
data_guten = get_report(url, headers_guten, yesterday_start, yesterday_end, 1, 'WB-GutenTech')
data_giper = get_report(url, headers_giper, yesterday_start, yesterday_end, 1, 'WB-ГиперМаркет')
data_kitchen = get_report(url, headers_kitchen, yesterday_start, yesterday_end, 1, 'WB-KitchenAid')
data_smart = get_report(url, headers_smart, yesterday_start, yesterday_end, 1, 'WB-Smart-Market')


# Function to flatten the JSON data for the current period
def flatten_json_current_period(cards):
    flattened_data = []
    for card in cards:
        nmID = card["nmID"]
        vendorCode = card["vendorCode"]
        brandName = card["brandName"]
        objectID = card["object"]["id"]
        objectName = card["object"]["name"]
        

        # Extract statistics for the selected period
        selected_period = card["statistics"]["selectedPeriod"]

        flattened_data.append({
            "nmID": nmID,
            "vendorCode": vendorCode,
            "brandName": brandName,
            "objectID": objectID,
            "objectName": objectName,
            "begin": selected_period["begin"],
            "end": selected_period["end"],
            "openCardCount": selected_period["openCardCount"],
            "addToCartCount": selected_period["addToCartCount"],
            "ordersCount": selected_period["ordersCount"],
            "ordersSumRub": selected_period["ordersSumRub"],
            "buyoutsCount": selected_period["buyoutsCount"],
            "buyoutsSumRub": selected_period["buyoutsSumRub"],
            "cancelCount": selected_period["cancelCount"],
            "cancelSumRub": selected_period["cancelSumRub"],
            "avgOrdersCountPerDay": selected_period["avgOrdersCountPerDay"],
            "avgPriceRub": selected_period["avgPriceRub"],
            "addToCartPercent": selected_period["conversions"]["addToCartPercent"],
            "cartToOrderPercent": selected_period["conversions"]["cartToOrderPercent"],
            "buyoutsPercent": selected_period["conversions"]["buyoutsPercent"],
            "stocksMp": card["stocks"]["stocksMp"],
            "stocksWb": card["stocks"]["stocksWb"]
        })

    return flattened_data

# Convert the flattened data to a DataFrame
flattened_data_guten = flatten_json_current_period(data_guten)
flattened_data_giper = flatten_json_current_period(data_giper)
flattened_data_kitchen = flatten_json_current_period(data_kitchen)
flattened_data_smart = flatten_json_current_period(data_smart)

df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)

# Add the 'Project' column to each DataFrame before concatenation
df_guten['Project'] = 'WB-GutenTech'
df_giper['Project'] = 'WB-ГиперМаркет'
df_kitchen['Project'] = 'WB-KitchenAid'
df_smart['Project'] = 'WB-Smart-Market'

# Combine all campaign data
combined_df = pd.concat([df_guten, df_giper, df_kitchen, df_smart], ignore_index=True)
combined_df['Marketplace'] = 'Wildberries'
print("Columns in combined_campaigns:", combined_df.columns.tolist())

# Display the DataFrame
combined_df


# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Page 1 retrieved successfully for WB-GutenTech (got 1000 items)
Page 2 retrieved successfully for WB-GutenTech (got 1000 items)
Page 3 retrieved successfully for WB-GutenTech (got 1000 items)
Rate limit exceeded for WB-GutenTech. Waiting 30 seconds...
Page 4 retrieved successfully for WB-GutenTech (got 1000 items)
Page 5 retrieved successfully for WB-GutenTech (got 1000 items)
Page 6 retrieved successfully for WB-GutenTech (got 1000 items)
Rate limit exceeded for WB-GutenTech. Waiting 30 seconds...
Page 7 retrieved successfully for WB-GutenTech (got 1000 items)
Page 8 retrieved successfully for WB-GutenTech (got 1000 items)
Rate limit exceeded for WB-GutenTech. Waiting 30 seconds...
Page 9 retrieved successfully for WB-GutenTech (got 1000 items)
Page 10 retrieved successfully for WB-GutenTech (got 840 items)
Reached last page for WB-GutenTech
Page 1 retrieved successfully for WB-ГиперМаркет (got 1000 items)
Page 2 retrieved successfully for WB-ГиперМаркет (got 115 items)
Reached last p

In [122]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [123]:
df_copy

,nmID,vendorCode,brandName,objectID,objectName,begin,end,openCardCount,addToCartCount,ordersCount,...,cancelSumRub,avgOrdersCountPerDay,avgPriceRub,addToCartPercent,cartToOrderPercent,buyoutsPercent,stocksMp,stocksWb,Project,Marketplace
0,159488075,I01475,AND,594,Массажеры электрические,2025-05-04 00:00:00,2025-05-04 23:59:59,754,59,12,...,0,12,4215,8,20,100,0,189,WB-GutenTech,Wildberries
1,110511376,Б0057093,Braun,634,Эпиляторы,2025-05-04 00:00:00,2025-05-04 23:59:59,95,7,2,...,0,2,19577,7,29,0,150,0,WB-GutenTech,Wildberries
2,198208484,4210201429395,braun,620,Бритвы электрические,2025-05-04 00:00:00,2025-05-04 23:59:59,104,8,3,...,0,3,10496,8,38,100,3,3,WB-GutenTech,Wildberries
3,43650894,Б0049647,Braun,648,Сетки для бритв,2025-05-04 00:00:00,2025-05-04 23:59:59,67,9,5,...,0,5,5290,13,56,100,0,13,WB-GutenTech,Wildberries
4,137609363,3014260331320,GILLETTE,1290,Кассеты для бритв,2025-05-04 00:00:00,2025-05-04 23:59:59,55,10,7,...,0,7,3105,18,70,0,1,0,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11182,288465484,8700216589017,GILLETTE,1527,Бритвы безопасные,2025-05-04 00:00:00,2025-05-04 23:59:59,38,1,0,...,0,0,0,3,0,0,0,59,WB-Smart-Market,Wildberries
11183,288465485,8700216588713,GILLETTE,1527,Бритвы безопасные,2025-05-04 00:00:00,2025-05-04 23:59:59,24,1,0,...,0,0,0,4,0,0,0,4,WB-Smart-Market,Wildberries
11184,288465487,8700216588911,GILLETTE,1527,Бритвы безопасные,2025-05-04 00:00:00,2025-05-04 23:59:59,20,0,0,...,0,0,0,0,0,0,0,71,WB-Smart-Market,Wildberries
11185,313543156,7702018085934,GILLETTE,1290,Кассеты для бритв,2025-05-04 00:00:00,2025-05-04 23:59:59,0,0,0,...,0,0,0,0,0,0,0,0,WB-Smart-Market,Wildberries


In [124]:
df_unique = df_copy['nmID'].unique()
print(df_unique)

[159488075 110511376 198208484 ... 288465487 313543156 313543513]


In [125]:
# Assuming your DataFrame is named df
specific_nmID = 137609342

# Filter the DataFrame for the specific nmID
filtered_df_id = df_copy[df_copy['nmID'] == specific_nmID]

# Group by nmID and sum the ordersSumRub
total_ordersSumRub = filtered_df_id['ordersSumRub'].sum()

print(f"Total ordersSumRub for nmID {specific_nmID}: {total_ordersSumRub}")

Total ordersSumRub for nmID 137609342: 4078


In [126]:
df_unique = df_copy['ordersSumRub'].sum()
print(df_unique)

2338492


In [127]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['day'] = pd.to_datetime(df_final_copy['day']).dt.date
df_copy['begin'] = pd.to_datetime(df_copy['begin']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'begin': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_15704\2097599021.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_15704\2097599021.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25088481.0,2025-05-04,Эпилятор женский электрический Silk-epil 9-880,424.0,27.0,94.06,0.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,1
1,25623718.0,2025-05-04,Бритва мужская электрическая S3 300BT с триммером,212.0,9.0,22.07,1.0,1.0,1.0,7858.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,7858,3
2,25623718.0,2025-05-04,Бритва мужская электрическая S3 300BT с триммером,350.0,17.0,71.39,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,1,7858,3
3,25624340.0,2025-05-04,Электробритва Series 3 300s,446.0,13.0,43.68,0.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,2,13090,8
4,25624340.0,2025-05-04,Электробритва Series 3 300s,393.0,24.0,80.08,5.0,2.0,2.0,13090.0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,2,13090,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,235597032.0,2025-05-04,Эпилятор женский электрический Silk-epil 7-030,3192.0,131.0,461.82,24.0,3.0,3.0,31914.0,...,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,2,21276,27
450,235599972.0,2025-05-04,Электробритва Series 9 Pro+ 9510s,563.0,16.0,63.52,3.0,2.0,2.0,59954.0,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,2,59954,3
451,235604574.0,2025-05-04,Триммер для бороды и усов BT5421,268.0,6.0,31.91,1.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,0,0,2
452,235638522.0,2025-05-04,Эпилятор Silk-epil 5 5-000,2691.0,101.0,353.95,16.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,1,7611,23


In [128]:
df_unique = merged_df_2['ordersSumRub'].sum()
print(df_unique)


1841929


## Insert the data

In [129]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [130]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                     float64
day               datetime64[ns]
name_product              object
views                    float64
clicks                   float64
sum                      float64
atbs                     float64
orders                   float64
shks                     float64
sum_price                float64
advertId                 float64
Project                   object
Marketplace               object
endTime                   object
createTime                object
startTime                 object
name_campaign             object
status                    object
type                      object
ordersCount                int64
ordersSumRub               int64
addToCartCount             int64
dtype: object
Sample data to insert: [(25088481.0, Timestamp('2025-05-04 00:00:00'), 'Эпилятор женский электрический Silk-epil 9-880', 424.0, 27.0, 94.06, 0.0, 0.0, 0.0, 0.0, 21204285.0, 'WB-GutenTech', 'Wildberries', datetime.date(2100, 1, 1), date